In [ ]:
!pip install streamlit
!pip install transformers
!pip install pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
import random
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datetime import datetime
import os

# Set your Hugging Face API key directly (replace with your actual API token)
hf_api_key = ""  # Replace with your Hugging Face API token

# Load GPT-2 model (lighter and more efficient)
model_name = "gpt2"  # You can replace with another model like gpt2-medium, gpt2-large, etc.

# Load model and tokenizer from Hugging Face
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_api_key)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_api_key)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Custom CSS for styling the Streamlit app
st.markdown("""
    <style>
    .stApp {
        background-color: #e0f7fa;
        font-family: 'Arial', sans-serif;
    }
    .header {
        color: #004d40;
        font-size: 45px;
        font-weight: bold;
        text-align: center;
        margin-top: 50px;
        text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);
    }
    .subheader {
        color: #00796b;
        font-size: 28px;
        font-weight: bold;
        margin-top: 20px;
    }
    .button {
        background-color: #00796b;
        color: white;
        font-size: 18px;
        border-radius: 10px;
        padding: 10px 20px;
    }
    .mood {
        background-color: #81c784;
        border-radius: 5px;
        padding: 10px;
        margin-top: 10px;
    }
    .container {
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100vh;
        text-align: center;
    }
    .footer {
        color: #004d40;
        font-size: 18px;
        text-align: center;
        padding: 20px;
    }
    .button:hover {
        background-color: #004d40;
    }
    /* Styling for stress relief advice */
    .response {
        font-size: 20px;
        color: #00796b;
        text-align: left;
        padding: 20px;
        margin-top: 20px;
        border-radius: 10px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        background-color: #ffffff;
        max-width: 80%;
        margin-left: 10%;
    }
    .thank-you {
        font-size: 22px;
        color: #00796b;
        background-color: #c8e6c9;
        padding: 20px;
        border-radius: 10px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        text-align: center;
    }
    .suggestion {
        font-size: 18px;
        color: #d32f2f;
        background-color: #ffcdd2;
        padding: 15px;
        border-radius: 8px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        text-align: center;
    }
    /* Styling for jokes */
    .joke {
        text-align: left;
        font-size: 18px;
        color: #00796b;
        background-color: #b2dfdb;
        padding: 15px;
        margin-top: 10px;
        border-radius: 8px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        margin-bottom: 20px;
    }
    </style>
""", unsafe_allow_html=True)

# Function to generate personalized stress-relief advice based on stress level and mood
def generate_stress_relief_advice(stress_level, mood):
    # Generating a concise and actionable tip using GPT-2 based on stress level and mood
    prompt = f"Based on the user's stress level of '{stress_level}' and mood of '{mood}', provide one short, actionable, and effective tip to help relieve stress."

    # Generate the tip using GPT-2
    generated_text = generator(prompt, max_length=100, num_return_sequences=1, temperature=0.7, top_k=50, top_p=0.85, no_repeat_ngram_size=2)[0]['generated_text']

    # Process the generated text to ensure it’s concise, without repetition, and without incomplete sentences
    processed_text = remove_incomplete_sentences(generated_text)
    return processed_text.strip()

# Function to remove incomplete sentences (those that don't end with a period)
def remove_incomplete_sentences(text):
    sentences = text.split('. ')
    # Remove the last sentence if it doesn't end with a period
    if sentences[-1] and sentences[-1][-1] not in ['.', '!', '?']:
        sentences = sentences[:-1]
    # Ensure all sentences are unique and return as a clean response
    seen = set()
    unique_sentences = []
    for sentence in sentences:
        if sentence not in seen:
            seen.add(sentence)
            unique_sentences.append(sentence)
    return '. '.join(unique_sentences) + '.' if unique_sentences else ''

# Function to provide jokes based on mood and stress level
def get_joke_based_on_mood_and_stress(stress_level, mood):
    # Stress-relief jokes for higher stress levels
    if stress_level == "Very Stressed 😫":
        jokes = [
            "Why don't skeletons ever fight each other? They don't have the guts! 😄",
            "I used to play piano by ear, but now I use my hands. 🎹",
            "Why don't oysters donate to charity? Because they are shellfish! 🦪",
            "Why did the bicycle fall over? It was two-tired! 🚲",
            "How does a penguin build its house? Igloos it together! 🐧",
        ]
    elif stress_level == "Moderately Stressed 😕":
        jokes = [
            "Why did the math book look sad? Because it had too many problems! 📚",
            "What do you call fake spaghetti? An impasta! 🍝",
            "I told my wife she was drawing her eyebrows too high. She looked surprised! 😂",
            "Why did the scarecrow win an award? Because he was outstanding in his field! 🌾",
            "I’m on a whiskey diet. I’ve lost three days already! 🥃",
        ]
    elif mood == "Happy 😄":
        jokes = [
            "I told my computer I needed a break, and now it won’t stop sending me kit-kats! 💻",
            "Why don’t skeletons fight each other? They don’t have the guts! 💀",
            "What’s orange and sounds like a parrot? A carrot! 🥕",
            "Why don’t eggs tell jokes? They’d crack each other up! 🥚",
            "I’m on a seafood diet... I see food and eat it! 🦞",
        ]
    elif mood == "Neutral 😐":
        jokes = [
            "Why was the math book sad? Because it had too many problems! 📚",
            "Why don’t skeletons fight each other? They don’t have the guts. 😆",
            "Why did the bicycle fall over? Because it was two-tired! 🚲",
            "How do you organize a space party? You planet! 🌍",
            "Why don’t seagulls fly over the bay? Because then they’d be bagels! 🥯",
        ]
    elif mood == "Tired 😴":
        jokes = [
            "I just burned 1200 calories. I forgot the pizza in the oven. 🍕",
            "I used to have a dog that was a cross between a bulldog and a shih tzu. It was a 'bullshih'! 🐕",
            "I’m not lazy, I’m just on energy-saving mode. 🔋",
            "Why do cows have hooves instead of feet? Because they lactose! 🐄",
            "I only work out on days that end in ‘y’. 🧘",
        ]
    else:
        jokes = ["Why don't skeletons fight each other? They don't have the guts! 👻"]

    return random.choice(jokes)

# Streamlit app layout
st.markdown('<div class="header">CalmCure: Stress-Free Living 🌿</div>', unsafe_allow_html=True)

# Step 1: Assess Your Mood First
mood = st.radio("How do you feel today?", ("Happy 😄", "Neutral 😐", "Stressed 😕", "Tired 😴"), index=0)

# Step 2: Assess Your Stress Level Next
stress_level = st.radio(
    "How stressed are you?",
    ("Not Stressed 😊", "Mildly Stressed 😐", "Moderately Stressed 😕", "Very Stressed 😫"),
    index=0,  # Default value
    help="Choose your current stress level."
)

# Step 3: Generate Stress Relief Advice and a Joke Based on Mood and Stress Level
st.markdown('<div class="subheader">Get Stress Relief Advice</div>', unsafe_allow_html=True)

if st.button("Get Stress Relief Advice 🧘", key="1"):
    advice = generate_stress_relief_advice(stress_level, mood)

    # Show advice
    st.markdown(f"<div class='response'>{advice}</div>", unsafe_allow_html=True)

# Step 4: Joke Section (Separate Button)
st.markdown('<div class="subheader">Get a Joke</div>', unsafe_allow_html=True)

if st.button("Get a Joke 🤣", key="2"):
    joke = get_joke_based_on_mood_and_stress(stress_level, mood)
    st.markdown(f"<div class='joke'>{joke}</div>", unsafe_allow_html=True)

# Step 5: Ask if it's better now
better_now = st.radio("Do you feel better now?", ("Yes 😊", "No 😞"))

if better_now == "Yes 😊":
    st.markdown("<div class='thank-you'>Thank you for using CalmCure! 😊 Stay stress-free and take care! 🌿</div>", unsafe_allow_html=True)
else:
    st.markdown("<div class='suggestion'>It seems like stress relief takes time! Work on the suggested exercises again and feel better. You got this! 💪</div>", unsafe_allow_html=True)

# Add Footer
st.markdown("""
    <footer class="footer">
        Made with ❤️ for Stress Relief
    </footer>
""", unsafe_allow_html=True)


Writing app.py


In [ ]:
from google.colab import files

files.download('app.py')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from pyngrok import ngrok
import os

# Set ngrok authtoken (replace with your own token)
ngrok.set_auth_token("")

# Open a tunnel to the streamlit app on port 8501
public_url = ngrok.connect(8501)

# Run the Streamlit app in the background
os.system('streamlit run app.py &')

# Print the public URL to access the app
print('Streamlit app is live at:', public_url)

Streamlit app is live at: NgrokTunnel: "https://primulaceous-overpainfully-krysten.ngrok-free.dev" -> "http://localhost:8501"
